In [13]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import MongoClient
from tqdm import tqdm
import time
from datetime import datetime

In [14]:
# MongoDB 클라이언트 및 컬렉션 설정
client = MongoClient(host="1.234.51.110", port=38019, username='clawling', password='goodtime**95')
db = client['weibo_crawling']
collection = db['OneAsia_weibo_crawling']

In [15]:
# 웹드라이버 초기화 및 weibo 검색 페이지 열기
driver = webdriver.Chrome()
driver.get("https://www.weibo.com")
driver.maximize_window()

In [16]:
# 버튼 요소 찾기
wait = WebDriverWait(driver, 10)
button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'LoginCard_btn_Jp_u1')))
    
# 버튼 클릭 (모바일 QR코드로 로그인, 30초 내로 완료해야 함.)
button.click()
time.sleep(30)

In [17]:
# 검색어 입력
search_input = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'input.woo-input-main')))
search_input.send_keys("부산원아시아페스티벌")

# 검색 버튼 요소 찾기
search_result_link = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.SearchBar_rWeibo_YdBa9 a.ALink_none_26qv1')))
search_result_link.click()

In [18]:
# 새 창으로 전환
original_window = driver.current_window_handle
for window_handle in driver.window_handles:
    if window_handle != original_window:
        driver.switch_to.window(window_handle)
        break

In [19]:
# 날짜 형식 설정 2023年11月03日 → 2023.11.03. or 11月03日 → 2024.11.03.
def convert_date_format(date_text):
    if '-' in date_text:
        # 하이픈(-)을 점(.)으로 변환하고 마지막에 점 추가
        formatted_date = date_text.replace('-', '.') + '.'
    else:
        # 年, 月, 日을 "."으로 변환
        formatted_date = date_text.replace("年", ".").replace("月", ".").replace("日", ".").strip()
        if formatted_date.count('.') == 3:
            return formatted_date
        # 연도가 누락된 경우는 올해 연도를 사용
        current_year = datetime.now().year
        return f"{current_year}.{formatted_date}"
    return formatted_date

In [20]:
SCROLL_PAUSE_TIME = 1.5
max_clicks = 48  # 최대 클릭 횟수 (weibo 검색 키워드 별 최대 페이지 수 확인 ex) 50p 라면 50-2=48, 48을 최대 클릭 횟수로 설정함.)
click_counter = 0  # 클릭 카운터 변수 초기화
all_data = []

while click_counter < max_clicks:
    try:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)

        # 채널명, URL, 업로드 날짜, 좋아요 개수 수집 
        user_names = driver.find_elements(By.CSS_SELECTOR, 'a.name')
        links = driver.find_elements(By.XPATH, '//div[@class="from"]/a[1]')
        date_elements = driver.find_elements(By.XPATH, '//div[@class="from"]/a[1]')
        like_elements = driver.find_elements(By.CSS_SELECTOR, 'div.card-act ul li a.woo-box-flex.woo-box-alignCenter.woo-box-justifyCenter span.woo-like-count')
    
        min_length = min(len(user_names), len(links), len(date_elements), len(like_elements))
    
        for i in range(min_length):
            user_name = user_names[i].text
            href = links[i].get_attribute("href")
            date_text = date_elements[i].text.split()[0]
            converted_date = convert_date_format(date_text)
            like_text = like_elements[i].text.strip()
            if like_text == "赞":
                count = 0
            elif like_text.isdigit():
                count = int(like_text)

            data = {
                "채널명": user_name,
                "URL": href,
                "업로드 날짜": converted_date,
                "좋아요": count,
                "검색키워드": "부산원아시아페스티벌"
            }
            all_data.append(data)
    
        # 다음 페이지로 이동
        next_page_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.next')))
        next_page_link.click()
        time.sleep(2)  # 페이지 이동 후 잠시 대기

        click_counter += 1  # 클릭 횟수 증가

    except Exception as e:
        print(f"예외 발생: {e}")
        break

# MongoDB에 데이터 적재
for data in tqdm(all_data, desc="데이터 적재 중"):
    collection.insert_one(data)

client.close()

데이터 적재 중: 100%|██████████| 47/47 [00:00<00:00, 85.00it/s]


In [21]:
driver.quit()